In [146]:
import pygsti
import numpy as np
from scipy.linalg import expm
from pygsti.circuits import Circuit
from matplotlib import pyplot as plt
from pygsti.processors import QuditProcessorSpec

# target model def

In [147]:
# Gell-Mann matrices
gellmann_matrices = [
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
    np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]),
    np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]]),
    np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]]),
    np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]]),
    np.array([[0, 0, -1j], [0, 0, 0], [1j, 0, 0]]),
    np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]),
    np.array([[0, 0, 0], [0, 0, -1j], [0, 1j, 0]]),
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, -2]])
]

gellmann_8_12 = np.array([[-2, 0, 0], [0, 1, 0], [0, 0, 1]])

In [175]:
# unitary models 
# we ignore axis error 
def modelX01(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[1] + gamma*gellmann_matrices[8]))

def modelZ01():
    return np.diag([np.exp(-1j*np.pi/4), 1, 1])

def modelZ12():
    return  np.diag([1, 1, np.exp(-1j*np.pi/4)])

X12_gen = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])

def modelX12(theta):
    return expm(-(1j/2)*((np.pi/2 + theta)*X12_gen))

def modelCZ(phis):
    return np.diag([1]+[np.exp(-1j*phi) for phi in phis])



In [176]:
circ_Y01_Q1 = [('Gz01', 'Q4')] + [('Gx01', 'Q4')] + [('Gz01', 'Q4')]*7
circ_Y12_Q1 = [('Gz12', 'Q4')] + [('Gx12', 'Q4')] + [('Gz12', 'Q4')]*7

circ_X01_Q1_inv = [('Gx01', 'Q4')]*3 + [('Gz_12', 'Q4')]*4
circ_X12_Q1_inv = [('Gx12', 'Q4')]*3 + [('Gz01', 'Q4')]*4
circ_Y01_Q1_inv = circ_Y01_Q1*3  + [('Gz12', 'Q4')]*4
circ_Y12_Q1_inv = circ_Y12_Q1*3 + [('Gz01', 'Q4')]*4

Gx01_pi_inv = [('Gx01', 'Q4')]*2 + [('Gz12', 'Q4')]*4
Gx12_pi_inv = [('Gx12', 'Q4')]*2 + [('Gz01', 'Q4')]*4

fiducial_prep_dict = {
    'id' : [],
    'X01_Q1_cos_prep': [], 
    'X01_Q1_sin_prep': [('Gx01', 'Q4'),], 
    'X12_Q1_cos_prep': [('Gx01', 'Q4'), ('Gx01', 'Q4')],
    'X12_Q1_sin_prep': [('Gx01', 'Q4'), ('Gx01', 'Q4'), ('Gx12', 'Q4')],
    'phase_Q1_cos_prep': [('Gx01', 'Q4'), ('Gx01', 'Q4')]  + circ_Y12_Q1 + circ_Y01_Q1 + [('Gz01', 'Q4')]*3 + [('Gz12', 'Q4')]*3,
    'phase_Q1_sin_prep': [('Gx01', 'Q4'), ('Gx01', 'Q4')]  + circ_Y12_Q1 + circ_Y01_Q1 + [('Gz01', 'Q4')]*3 + [('Gz12', 'Q4')]*1,
}

fiducial_meas_dict = {
    'id' : [],
    'X01_Q1_cos_meas': [], 
    'X01_Q1_sin_meas': [],
    'X12_Q1_cos_meas': [],
    'X12_Q1_sin_meas': [],
    'phase_Q1_cos_meas': [('Gz01', 'Q4')]*5 + [('Gz12', 'Q4')]*5 + circ_Y01_Q1_inv + circ_Y12_Q1_inv + Gx01_pi_inv,
    'phase_Q1_sin_meas': [('Gz01', 'Q4')]*5 + [('Gz12', 'Q4')]*5 + circ_Y01_Q1_inv + circ_Y12_Q1_inv + Gx01_pi_inv,
}

In [177]:
def make_single_qutrit_rpe_edesign_Q1(depths):
    circuits = {
        'X01' : {'cos': [], 'sin': []},
        'X01' : {'cos': [], 'sin': []},
        'X12' : {'cos': [], 'sin': []},
        'X12' : {'cos': [], 'sin': []},
        'phase' : {'cos': [], 'sin': []},
        'phase' : {'cos': [], 'sin': []},
    }
    for depth in depths:
        circuits['X01']['cos'].append(make_rpe_circuit([('Gx01', 'Q4')], 'X01_Q1_cos_prep', 'X01_Q1_cos_meas', depth))
        circuits['X01']['sin'].append(make_rpe_circuit([('Gx01', 'Q4')], 'X01_Q1_sin_prep', 'X01_Q1_sin_meas', depth))
        circuits['X12']['cos'].append(make_rpe_circuit([('Gx12', 'Q4')], 'X12_Q1_cos_prep', 'X12_Q1_cos_meas', depth))
        circuits['X12']['sin'].append(make_rpe_circuit([('Gx12', 'Q4')], 'X12_Q1_sin_prep', 'X12_Q1_sin_meas', depth))
        circuits['phase']['cos'].append(make_rpe_circuit([('Gx01', 'Q4')], 'phase_Q1_cos_prep', 'phase_Q1_cos_meas', depth))
        circuits['phase']['sin'].append(make_rpe_circuit([('Gx01', 'Q4')], 'phase_Q1_sin_prep', 'phase_Q1_sin_meas', depth))
    return circuits

In [178]:
def make_rpe_circuit(gate, prep_label, meas_label, depth):
    prep_circ = fiducial_prep_dict[prep_label]
    meas_circ = fiducial_meas_dict[meas_label]
    return Circuit(prep_circ + gate*depth + meas_circ)

In [179]:
def edesign_to_circuit_list(edesign):
    circuits = []
    for etype in edesign.keys():
        for mtype in edesign[etype].keys():
            for circuit in edesign[etype][mtype]:
                circuits.append(circuit)
    return pygsti.tools.remove_duplicates(circuits)

In [199]:
2**10

1024

In [200]:
edesign = make_single_qutrit_rpe_edesign_Q1([2**i for i in range(10)])
circ_list = edesign_to_circuit_list(edesign)
pygsti.io.write_circuit_list('single_qutrit.txt', circ_list)

In [181]:
edesign

{'X01': {'cos': [Circuit(Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4))],
  'sin': [Circuit(Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4))]},
 'X12': {'cos': [Circuit(Gx01:Q4Gx01:Q4Gx12:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4@(Q4)),
   Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q

In [182]:
circ_list

[Circuit(Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx12:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4Gx12:Q4@(Q4)),
 Circuit(Gx01:Q4Gx01

In [183]:
from quapack.pyRPE import RobustPhaseEstimation
from quapack.pyRPE.quantum import Q as _rpeQ

def estimate_phase_from_dataset(ds, cos_circs, cos_outcome0s, cos_outcome1s, sin_circs, sin_outcome0s, sin_outcome1s, depths):
    experiment = _rpeQ()
    for idx, d in enumerate(depths):
        if d == 0:
            continue
        cos_0_counts = sum([ds[cos_circs[idx]].counts[ci] for ci in cos_outcome0s])
        cos_1_counts = sum([ds[cos_circs[idx]].counts[ci] for ci in cos_outcome1s])
        sin_0_counts = sum([ds[sin_circs[idx]].counts[ci] for ci in sin_outcome0s])
        sin_1_counts = sum([ds[sin_circs[idx]].counts[ci] for ci in sin_outcome1s])
        experiment.process_cos(d, (int(cos_0_counts), int(cos_1_counts)))
        experiment.process_sin(d, (int(sin_0_counts), int(sin_1_counts)))
    analysis = RobustPhaseEstimation(experiment)
    last_good_generation = analysis.check_unif_local(historical=True)
    estimates = analysis.angle_estimates
    return estimates, last_good_generation

In [184]:
from pygsti.baseobjs import ExplicitStateSpace
from pygsti.models import ExplicitOpModel
from pygsti.models.modelconstruction import create_spam_vector
from pygsti.modelmembers.povms import UnconstrainedPOVM, FullPOVMEffect
from pygsti.modelmembers.states import FullState
from pygsti.tools import change_basis
from pygsti.baseobjs import Basis


def make_model(error_vector, depolarization, qid):
    
    # Parse error vector
    x01_overrot = error_vector[0]
    x12_overrot = error_vector[1]
    phase_error = error_vector[2]

    # Define single qutrit unitaries
    X01_unitary = modelX01(x01_overrot, phase_error)
    Z01_unitary = modelZ01()
    X12_unitary = modelX12(x12_overrot)
    Z12_unitary = modelZ12()


    target_unitary_mapping = {
        'Gx01': X01_unitary,
        'Gx12': X12_unitary,
        'Gz01': Z01_unitary,
        'Gz12': Z12_unitary,
    }

    rho0 = np.asarray([1, 0, 0])

    E0 = np.asarray([1, 0, 0])
    E1 = np.asarray([0, 1, 0])
    E2 = np.asarray([0, 0, 1])

    povm_dict = {'0' : E0, '1' : E1, '2' : E2}

    
    gate_names = ['Gx01', 'Gx12', 'Gz01', 'Gz12']

    availability = {
        'Gx01': [(qid, )], 
        'Gx12': [(qid, )], 
        'Gz01': [(qid, )], 
        'Gz12': [(qid, )], 
    }


    # define the processor spec
    pspec = QuditProcessorSpec([qid,], qudit_udims=[3], gate_names=gate_names,
                    nonstd_gate_unitaries=target_unitary_mapping,
                    prep_names = ['rho0'], povm_names = ['Mdefault'],
                    nonstd_preps = {'rho0': rho0}, nonstd_povms = {'Mdefault': povm_dict},
                    availability=availability,
                    )
    model = pygsti.models.modelconstruction.create_explicit_model(pspec,ideal_gate_type='full TP', ideal_spam_type='full pure', basis='gm')

    model.operations[('Gx01', qid)].depolarize(depolarization)
    model.operations[('Gz01', qid)].depolarize(depolarization)
    model.operations[('Gx12', qid)].depolarize(depolarization)
    model.operations[('Gz12', qid)].depolarize(depolarization)
    return model

In [185]:
circ_list[-1]

Circuit(Gx01:Q4Gx01:Q4Gz12:Q4Gx12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz01:Q4Gx01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz12:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz01:Q4Gx01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gx01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gx01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gx12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gx12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gx12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gz01:Q4Gx01:Q4Gx01:Q4Gz12:Q4Gz12:Q4Gz12:Q4Gz12:Q4@(Q4))

In [186]:
ds_experimental = pygsti.io.load_dataset('qutrit_RPE_test_dataset.txt', cache=True)

Reading from cache file: qutrit_RPE_test_dataset.txt.cache


/opt/anaconda3/envs/qubic/lib/python3.12/site-packages/pygsti/tools/legacytools.py:38: UserWarning: The function load_dataset is deprecated, and may not be present in future versions of pygsti.
    Please use read_dataset instead.
  _warnings.warn(message)


In [187]:
print(ds_experimental)

Gx01:Q4@(Q4)  :  {('0',): 459.0, ('1',): 440.0, ('2',): 2.0}
Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 22.0, ('1',): 877.0, ('2',): 2.0}
Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 896.0, ('1',): 5.0, ('2',): 0.0}
Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 894.0, ('1',): 7.0, ('2',): 0.0}
Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 896.0, ('1',): 5.0, ('2',): 0.0}
Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 455.0, ('1',): 444.0, ('2',): 2.0}
Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 467.0, ('1',): 433.0, ('2',): 1.0}
Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 446.0, ('1',): 455.0, ('2',): 0.0}
Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4Gx01:Q4@(Q4)  :  {('0',): 418.0, ('1',): 482.0, ('2',): 1.0}
Gx01:Q4Gx01:Q4Gx12:Q4@(Q4)  :  {('0',): 15.0, ('1',): 470.0, ('2',

In [188]:
cos_circs_X01 = edesign['X01']['cos']  
sin_circs_X01 = edesign['X01']['sin']
cos_outcome0s = ['0',]
cos_outcome1s = ['1',]
sin_outcome0s = ['0',]
sin_outcome1s = ['1',]

In [189]:
estimate_phase_from_dataset(ds_experimental, cos_circs_X01, cos_outcome0s, cos_outcome1s, sin_circs_X01, sin_outcome0s, sin_outcome1s, [1, 2, 4, 8, 16])

(array([4.73460755, 4.70595659, 4.72193478, 4.7111207 , 4.70790238]), 4)

In [190]:
estimate_phase_from_dataset(ds_experimental, cos_circs_X01, cos_outcome0s, cos_outcome1s, sin_circs_X01, sin_outcome0s, sin_outcome1s, [1, 2, 4, 8, 16])

(array([4.73460755, 4.70595659, 4.72193478, 4.7111207 , 4.70790238]), 4)

In [191]:
cos_circs_X12 = edesign['X12']['cos']
sin_circs_X12 = edesign['X12']['sin']
cos_outcome0s = ['1',]
cos_outcome1s = ['2',]
sin_outcome0s = ['1',]
sin_outcome1s = ['2',]

In [192]:
estimate_phase_from_dataset(ds_experimental, cos_circs_X12, cos_outcome0s, cos_outcome1s, sin_circs_X12, sin_outcome0s, sin_outcome1s, [1, 2, 4, 8, 16])


(array([4.77779638, 4.72932595, 4.75269856, 4.74055286, 4.73636375]), 4)

In [193]:
ds_experimental[circ_list[-1]]

In [196]:
cos_circs_phase = edesign['phase']['cos']
sin_circs_phase = edesign['phase']['sin']
cos_outcome0s = ['0',]
cos_outcome1s = ['2',]
sin_outcome0s = ['0',]
sin_outcome1s = ['2',]


In [197]:
estimate_phase_from_dataset(ds_experimental, cos_circs_phase, cos_outcome0s, cos_outcome1s, sin_circs_phase, sin_outcome0s, sin_outcome1s, [1, 2, 4, 8, 16])

(array([2.31679695, 3.68972141, 2.94715463, 2.65089834, 2.50222933]), 1)

In [145]:
ds_experimental.keys()

<generator object DataSet.keys at 0x293c7c340>